In [1]:
import torch
import torch.nn as nn
import math
import nltk
from torch.utils.data import Dataset, DataLoader
from tokenizers import ByteLevelBPETokenizer

In [2]:
# nltk.download('punkt_tab')

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}!")

Using cuda!


# Prepare Data

In [4]:
# Train
path_to_data = "data/wiki.train.tokens/wiki.train.tokens"

In [5]:
# Train a tokenizer from your text files
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=[path_to_data], vocab_size=30_000, min_frequency=2, special_tokens=['<SOS>', '<EOS>', '<PAD>', '<UNK>'])

In [6]:
# Train
all_data = []
with open(path_to_data, 'r', encoding="utf-8") as f:
    curr = []
    for line in f:
        line = line.strip()
        
        if not line:
            continue

        elif line[0] == '=':
            all_data.append(' '.join(curr))
            curr = []
        else:
            curr.append(line)

wiki_tokens = [tokenizer.encode(line).ids for line in all_data]

wiki_tokens.sort(key=lambda x: len(x))
train_tokens = [s for s in wiki_tokens if s]

In [7]:
# Val
path_to_data = "data/wiki.valid.tokens/wiki.valid.tokens"

all_data = []
with open(path_to_data, 'r', encoding="utf-8") as f:
    curr = []
    for line in f:
        line = line.strip()
        
        if not line:
            continue

        elif line[0] == '=':
            all_data.append(' '.join(curr))
            curr = []
        else:
            curr.append(line)

wiki_tokens = [tokenizer.encode(line).ids for line in all_data]

wiki_tokens.sort(key=lambda x: len(x))
val_tokens = [s for s in wiki_tokens if s]

In [ ]:
print([tokenizer.decode(train_tokens[-5])])
print([tokenizer.decode(val_tokens[-5])])

In [6]:
pad_idx = tokenizer.encode("<PAD>").ids[0]
pad_idx

2

In [ ]:
def process(sentence):
    sos_idx = tokenizer.encode('<SOS>').ids[0]
    eos_idx = tokenizer.encode('<EOS>').ids[0]

    return [sos_idx] + sentence + [eos_idx]

def decode(sentence):
    return tokenizer.decode(list(sentence.detach().cpu()))

def add_padding(X, y, pad_idx, batch_size=32):
    num_sentences = len(X)
    for i in range(0, num_sentences, batch_size):
        idx = min(num_sentences-1, i + batch_size-1)
        max_length = len(X[idx])
        for j in range(i, idx+1):
            missing_X = max_length - len(X[j])
            
            X[j].extend([pad_idx]*missing_X)
            y[j].extend([pad_idx]*(missing_X + 1))

    return X, y

class LMDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.long), torch.tensor(self.y[idx], dtype=torch.long)

batch_size = 16

# Train dataloader
train_tokens = [process(tokens) for tokens in train_tokens]
y_tokens = [tokens[1:] for tokens in train_tokens]
X_train, y_train = add_padding(train_tokens, y_tokens, pad_idx, batch_size)
train_dataset = LMDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Val dataloader
val_tokens = [process(tokens) for tokens in val_tokens]
y_tokens = [tokens[1:] for tokens in val_tokens]
X_val, y_val = add_padding(val_tokens, y_tokens, pad_idx, batch_size)
val_dataset = LMDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [8]:
vocabulary_size = 30000

In [ ]:
vocabulary_size = len(tokenizer.get_vocab())
print(f"Size of vocabulary: {vocabulary_size}\n")

for X, y in train_loader:
    print(X.shape)
    print(y.shape)
    break

print()

for X, y in val_loader:
    print(X.shape)
    print(y.shape)
    break

# Architecture

In [31]:
class MultiAttentionHead(nn.Module):
    def __init__(self, num_heads, hidden_size, dropout=0.1):
        super().__init__()
        head_dim = hidden_size // num_heads

        # Weight matrices for queries, keys, values
        self.W_q = nn.Parameter(torch.randn(num_heads, hidden_size, head_dim))
        self.W_k = nn.Parameter(torch.randn(num_heads, hidden_size, head_dim))
        self.W_v = nn.Parameter(torch.randn(num_heads, hidden_size, head_dim))

        self.hidden_size = hidden_size
        self.head_dim = head_dim
        self.num_heads = num_heads

        # Store the key and values
        self.past_key = None 
        self.past_value = None 

        # Output linear projection
        self.output_proj = nn.Parameter(torch.randn(hidden_size, hidden_size))

        # Normalization layer
        self.norm = nn.LayerNorm(hidden_size)

        # Attention dropout
        self.attention_dropout = nn.Dropout(dropout)

        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):        
        std = 0.02  
        
        with torch.no_grad():
            self.W_q.normal_(0, std)
            self.W_k.normal_(0, std) 
            self.W_v.normal_(0, std)
            self.output_proj.normal_(0, std)

    def forward(self, X, padding_mask, auto_reg=False):
        """
        Forward pass for multi-head attention.

        Inputs:
            X            : (batch_size, seq_len, hidden_size)
            padding_mask : (batch_size, seq_len)  -> 1 for real tokens, 0 for padding
        """

        batch_size, seq_len, _ = X.shape
        device = X.device 
        X_heads = X.unsqueeze(1)

        if not auto_reg:
            # Create padding mask
            mask_matrix = padding_mask.unsqueeze(-1) & padding_mask.unsqueeze(-2)
            pad_mask = mask_matrix.unsqueeze(1).expand(-1, self.num_heads, -1, -1)

            # Create causal mask
            causal_mask = torch.tril(torch.ones((seq_len, seq_len), device=device, dtype=bool))
            causal_mask = causal_mask.unsqueeze(0).unsqueeze(0).expand(batch_size, self.num_heads, -1, -1)

            # Combine masks
            att_mask = pad_mask & causal_mask

            # Add head dimension to input
            X_heads = X.unsqueeze(1)  # (batch, 1, seq_len, hidden_size)

            # Compute Q, K, V for full sequence
            Q = torch.matmul(X_heads, self.W_q)  # (batch, num_heads, seq_len, head_dim)
            K = torch.matmul(X_heads, self.W_k)  # (batch, num_heads, seq_len, head_dim)
            V = torch.matmul(X_heads, self.W_v)  # (batch, num_heads, seq_len, head_dim)
            
        else:
    
            # Compute Q for current token only
            Q = torch.matmul(X_heads, self.W_q)  # (batch, num_heads, 1, head_dim)
            
            if self.past_key is None:
                # First token - compute K,V for the entire sequence so far
                K = torch.matmul(X_heads, self.W_k)
                V = torch.matmul(X_heads, self.W_v)
            
            else:
                # Subsequent tokens - compute K,V only for current token and concatenate
                curr_K = torch.matmul(X_heads, self.W_k)  # (batch, num_heads, 1, head_dim)
                curr_V = torch.matmul(X_heads, self.W_v)
                
                # Concatenate with cached values
                K = torch.cat([self.past_key, curr_K], dim=2)
                V = torch.cat([self.past_value, curr_V], dim=2)

            self.past_key = K
            self.past_value = V


        # Compute attention scores
        K_transposed = K.transpose(-2, -1)  # (batch, num_heads, head_dim, kv_seq_len)
        scores = torch.matmul(Q, K_transposed)  # (batch, num_heads, q_seq_len, kv_seq_len)
        scaled_scores = scores / (self.head_dim ** 0.5)

        # Apply mask and softmax
        LARGE_NEG = -1e9
        if not auto_reg:
            scaled_scores = scaled_scores.masked_fill(~att_mask, LARGE_NEG)
        
        att_weights = torch.softmax(scaled_scores, dim=-1)
        att_weights = self.attention_dropout(att_weights)

        # Attention output
        att_output = torch.matmul(att_weights, V)  # (batch, num_heads, q_seq_len, head_dim)
        
        if auto_reg:
            combined_heads = att_output.reshape(batch_size, 1, self.hidden_size)
        else:
            combined_heads = att_output.permute(0, 2, 1, 3).contiguous().view(batch_size, seq_len, self.hidden_size)

        # Apply layer normalization and the final projection
        norm_output = self.norm(combined_heads)
        projected_output = torch.matmul(norm_output, self.output_proj)

        return projected_output
    
    def clear_cache(self):
        self.past_key, self.past_value = None, None

class FFN(nn.Module):
    def __init__(self, hidden_layers=2, act='relu', hidden_size=[32, 32], input_size=32, dropout=0.1):
        super().__init__()
        layers = [nn.Linear(input_size, hidden_size[0])]

        for i in range(1, hidden_layers):
            if act == 'relu':
                layers.append(nn.ReLU())

            elif act == 'tanh':
                layers.append(nn.Tanh())
            
            layers.append(nn.Dropout(dropout))
            layers.append(nn.Linear(hidden_size[i-1], hidden_size[i]))
            
                
        self.layers = nn.Sequential(*layers)
        

    def forward(self, X):
        for layer in self.layers:
            X = layer(X)

        return X

class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_seq_len, dropout=0.3):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
    
        position = torch.arange(max_seq_len).unsqueeze(1) # (seq_len, 1)
        div_term = torch.exp(torch.arange(0, embed_size, 2) * (-math.log(10000.0) / embed_size))
        pe = torch.zeros(1, max_seq_len, embed_size)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term) # (1, max_seq_len, embed_size)
        self.register_buffer('pe', pe) # So it goes to CPU/GPU as well

    def forward(self, X):
        """
        Arguments:
            X (batch_size, seq_len, embed_size)
        """
        X = X + self.pe[:, :X.shape[1],:]
        return self.dropout(X)  

class Transformer(nn.Module):
    def __init__(self, voc_size, embed_size, num_heads, depth, pad_idx, p=0.5):
        super().__init__()
        self.voc_size = voc_size
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.depth = depth
        self.pad_idx = pad_idx
        
        # Embeddings layer
        self.embed = nn.Embedding(voc_size, embed_size)
        nn.init.xavier_uniform_(self.embed.weight)
        self.embed_dropout = nn.Dropout(p*0.5)

        # Positional Encoding
        self.pe = PositionalEncoding(embed_size=self.embed_size, max_seq_len=2050, dropout=p).to(device)

        # Dropout layer
        self.dropout = nn.Dropout(p)

        # Attention Heads
        self.heads = nn.ModuleList([
            nn.ModuleList([
                MultiAttentionHead(num_heads, embed_size),
                nn.LayerNorm(embed_size),
                FFN(hidden_layers=2, act="relu",
                    hidden_size=[4 * embed_size, embed_size],
                    input_size=embed_size, dropout=p*0.8),
                nn.LayerNorm(embed_size)
            ])
            for _ in range(depth)
        ])

        # Last linear layer
        self.linear = nn.Linear(embed_size, voc_size, bias=False)
    

    def forward(self, X, auto_reg=False):
        '''
        Inputs:
            X (batch_size, seq_length)
        '''
        _, seq_len = X.shape
        padding_mask = X != self.pad_idx

        # Positional Encoding

        X = self.embed(X) # (batch_size, seq_length, embedding_size)
        X = self.embed_dropout(X)
        X = self.pe(X)


        for head, norm1, ffn, norm2 in self.heads:
            # Go through the multi heads
            X = self.dropout(norm1(head(X, padding_mask, auto_reg) + X))

            # Do the same for the FFN
            X = self.dropout(norm2(ffn(X) + X))
        
        output = self.linear(X)
        
        return output
    
    def generate_from_prompt(self, inputs, sample='p', max_size=1000, temperature=0.7):
        self.eval()
        self.clear_memo()
        curr = len(inputs)
        eos_idx = tokenizer.encode("<EOS>").ids[0]
        unk_idx = tokenizer.encode("<UNK>").ids[0]

        # Warmup
        if len(inputs) > 1:
            with torch.no_grad():
                for token in inputs[:-1]:
                    X = torch.tensor([[token]], dtype=torch.long).to(device)
                    _ = self(X, auto_reg=True)  
        
        while curr < max_size:
            X = torch.tensor([[inputs[-1]]], dtype=torch.long).to(device)  
            

            with torch.no_grad():
                logits = self(X, auto_reg=True)
                logits = logits / temperature
            
            match sample:
                case 'k':
                    token = self.top_k_sample(logits)[0, 0].item()
                case 'greedy':
                    token = self.greedy_sample(logits)[0, 0].item()
                case 'p':
                    token = self.top_p_sample(logits)[0, 0].item()

            if token == eos_idx:
                break
            elif token == unk_idx:
                continue

            curr += 1
            inputs.append(token)
        
        return tokenizer.decode(inputs, skip_special_tokens=True)

    def clear_memo(self):
        for module in self.modules():
            if isinstance(module, MultiAttentionHead):
                module.clear_cache()


    def greedy_sample(self, output):
        with torch.no_grad():
            return torch.argmax(output, dim=-1)


    def top_k_sample(self, output, k=20):
        '''
        Inputs:
            output (batch_size, seq_len, voc_size)
        '''
        device = output.device
        batch_size, seq_len, _ = output.shape 
        res = torch.zeros((batch_size, seq_len), dtype=torch.long, device=device)
        with torch.no_grad():
            probabilities = torch.softmax(output, dim=-1)
            values, indices = torch.topk(probabilities, k=k, dim=-1)
            batch_idx = torch.arange(batch_size).to(device)
            for pos in range(seq_len): 
                sample = torch.multinomial(values[:, pos, :], num_samples=1).squeeze(1) # (batch_size)
                res[:, pos] = indices[batch_idx, pos, sample]
        
        return res

    def top_p_sample(self, output, p=0.7):
        '''
        Inputs:
            output (batch_size, seq_len, voc_size)
        '''
        batch_size, seq_len, _ = output.shape
        device = output.device
        res = torch.zeros((batch_size, seq_len), dtype=torch.long, device=device)
        
        with torch.no_grad():
            probs = torch.softmax(output, dim=-1)  # (batch, seq_len, vocab)
            batch_idx = torch.arange(batch_size).to(device)
            sorted_probs, sorted_idx = torch.sort(probs, descending=True, dim=-1)
            cum_probs = sorted_probs.cumsum(dim=-1)
            mask = cum_probs > p
            mask[..., 1:] = mask[..., :-1].clone()  # keep at least first token
            mask[..., 0] = 0
            sorted_probs[mask] = 0.0
            sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)
            for pos in range(seq_len):
                sample = torch.multinomial(sorted_probs[:, pos, :], num_samples=1).squeeze(-1)
                res[:, pos] = sorted_idx[batch_idx, pos, sample]

        return res

In [32]:
# Example dimensions
batch_size = 2    # number of sequences
seq_len = 5       # tokens per sequence
hidden_size = 512   # embedding dimension
h = 8 # number of heads
head = MultiAttentionHead(h, hidden_size).to(device)

# Create random input
X = torch.randn(batch_size, seq_len, hidden_size).to(device)
att_mask = (torch.randint(0, 2, (batch_size, seq_len)) == 1).to(device)
output = head(X, att_mask)

print("Shape of X:", X.shape)
print(output.shape)

Shape of X: torch.Size([2, 5, 512])
torch.Size([2, 5, 512])


In [33]:
transformer = model = Transformer(voc_size=vocabulary_size, embed_size=128, num_heads=4, depth=4,\
                    pad_idx=pad_idx, p=0.6).to(device)

state = torch.load("models/overall_best.pth", map_location=device, weights_only=True)
transformer.load_state_dict(state)

<All keys matched successfully>

In [ ]:
prompt = "Playstation"
ans = transformer.generate_from_prompt(tokenizer.encode(prompt).ids, sample='k')
ans

In [81]:
transformer = Transformer(voc_size=vocabulary_size, embed_size=256, num_heads=4, depth=6,\
                    pad_idx=pad_idx, p=0.35).to(device)
X_test = torch.tensor(X_train[:10]).to(device)
#transformer = Transformer(voc_size=len(word2idx), embed_size=512, num_heads=8, depth=12,\
                          #pad_idx=word2idx['<PAD>'], p=0.3).to(device)

output = transformer(X_test)

prediction_k = transformer.top_k_sample(output)
prediction_greedy = transformer.greedy_sample(output)
prediction_p = transformer.top_p_sample(output)
print(f"Output shape of k sampling: {prediction_k.shape}")
print(f"Output shape of greedy sampling: {prediction_greedy.shape}")
print(f"Output shape of greedy sampling: {prediction_p.shape}")

Output shape of k sampling: torch.Size([10, 4])
Output shape of greedy sampling: torch.Size([10, 4])
Output shape of greedy sampling: torch.Size([10, 4])


In [ ]:
print("Text for p sampling:")
for batch in prediction_p:
    print(decode(batch))

print("\nText for k sampling:")
for batch in prediction_k:
    print(decode(batch))

print("\nText for greedy sampling:")

for batch in prediction_greedy:
    print(decode(batch))

In [83]:
def train_one_epoch(model, train_loader, optimizer, scheduler, criterion):
    model.train()
    total_loss = 0.0
    for i, (batch_X, batch_y) in enumerate(train_loader):
        if (i / len(train_loader)) > 0.5:
            break
            print(f"{(i / len(train_loader)) * 100:.2f}% done")
            print(total_loss)
            print(total_loss/(i+1))

        optimizer.zero_grad()

        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device) # (batch_size, seq_len)
        
        outputs = model(batch_X) # (batch_size, seq_len, voc_size)
        outputs = outputs.permute(0, 2, 1) # (batch_size, voc_size, seq_len) -> expected size for nn.CrossEntropy

        loss = criterion(outputs, batch_y)
        loss.backward()


        nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

        optimizer.step()

        total_loss += loss.item() 

    scheduler.step()

    return total_loss / len(train_loader)

def evaluate(model, val_loader, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for i, (batch_X, batch_y) in enumerate(val_loader):
            if i % 3000 == 0 or i == len(train_loader) - 1:
                print(f"{(i / len(train_loader)) * 100:.2f}% done")

            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device) # (batch_size, seq_len)
            
            outputs = model(batch_X) # (batch_size, seq_len, voc_size)
            outputs = outputs.permute(0, 2, 1) # (batch_size, voc_size, seq_len) -> expected size for nn.CrossEntropy

            loss = criterion(outputs, batch_y)

            total_loss += loss.item() 

    return total_loss / len(val_loader)

In [84]:
def setup_transformer_training(model, total_steps, warmup_steps):
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=3e-4,             
        weight_decay=0.01,
        betas=(0.9, 0.98)
    )

    # Learning rate schedule with warmup + linear decay to zero
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lr_lambda=lambda step: min((step + 1) / warmup_steps, 
                                   max(0.0, (total_steps + warmup_steps - step) / max(1, total_steps)))
    )

    return optimizer, scheduler

In [85]:
num_epochs = 50
warmup_epochs = 5
patience = 5
model = Transformer(voc_size=vocabulary_size, embed_size=128, num_heads=4, depth=4,\
                    pad_idx=pad_idx, p=0.2).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx, label_smoothing=0.05)

opt, scheduler = setup_transformer_training(model, num_epochs, warmup_epochs)

best_loss = float('inf')
best_model_state = None
curr = 0

for epoch in range(1, num_epochs+1):
    print("-"*50)
    print(f"Epoch {epoch}")
    train_loss = train_one_epoch(model, train_loader, opt, scheduler, criterion)
    val_loss = evaluate(model, val_loader, criterion)
    print(f"Train Loss: {train_loss:.5f}")
    print(f"Val Loss: {val_loss:.5f}")
    
    # Early stopping logic
    if val_loss < best_loss:
        best_loss = val_loss
        best_model_state = model.state_dict()
        print(f"Found better model at epoch {epoch}. Saving...")
        torch.save(best_model_state, "models/best_model.pth")
        curr = 0
    else:
        curr += 1
        if curr == patience:
            print("No more patience. Ending Training Loop...")
            break

    print(f"Best Val Loss: {best_loss:.5f}")
    print("-"*50)

# If it ends the training loop
torch.save(best_model_state, "models/overall_best.pth")  

--------------------------------------------------
Epoch 1
0.00% done
Train Loss: 3.87280
Val Loss: 7.38937
Found better model at epoch 1. Saving...
Best Val Loss: 7.38937
--------------------------------------------------
--------------------------------------------------
Epoch 2
0.00% done
Train Loss: 3.53126
Val Loss: 6.73738
Found better model at epoch 2. Saving...
Best Val Loss: 6.73738
--------------------------------------------------
--------------------------------------------------
Epoch 3
0.00% done
Train Loss: 3.38861
Val Loss: 6.50789
Found better model at epoch 3. Saving...
Best Val Loss: 6.50789
--------------------------------------------------
--------------------------------------------------
Epoch 4
0.00% done
Train Loss: 3.31570
Val Loss: 6.47488
Found better model at epoch 4. Saving...
Best Val Loss: 6.47488
--------------------------------------------------
--------------------------------------------------
Epoch 5
0.00% done
Train Loss: 3.26680
Val Loss: 6.50967
